# Hello World - NLP Example

Start by importing nuclio to the notebook to enable nuclio's "Magic Commands".  
Since we don't want nuclio itself to be imported to the function we will use the comment annotation: ``` # nuclio: ignore```.  

This will let nuclio know that we do not want this cell included in the function

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

# Define environment

## Build commands

In [2]:
%nuclio cmd -c pip3 install textblob

## Environment variables

In [3]:
%nuclio env -l TO_LANG=fr
%nuclio env -c TO_LANG=es

%nuclio: setting 'TO_LANG' environment variable


In [4]:
from textblob import TextBlob
import os

def handler(context, event):
    context.logger.info('This is an NLP example! ')

    # process and correct the text
    blob = TextBlob(str(event.body.decode('utf-8')))
    corrected = blob.correct()

    # debug print the text before and after correction
    context.logger.info_with("Corrected text", corrected=str(corrected), orig=str(blob))

    # calculate sentiments
    context.logger.info_with("Sentiment",
                             polarity=str(corrected.sentiment.polarity),
                             subjectivity=str(corrected.sentiment.subjectivity))

    # read target language from environment and return translated text
    lang = os.getenv('TO_LANG','fr')
    return str(corrected.translate(to=lang))

In [5]:
# nuclio: end-code

In [6]:
# nuclio: ignore
event = nuclio.Event(body=b"Have a great day with your friends!")
handler(context, event)

Python> 2019-11-17 10:33:02,134 [info] This is an NLP example! 
Python> 2019-11-17 10:33:02,188 [info] Corrected text: {'corrected': 'Have a great day with your friends!', 'orig': 'Have a great day with your friends!'}
Python> 2019-11-17 10:33:02,236 [info] Sentiment: {'polarity': '1.0', 'subjectivity': '0.75'}


'Passez une bonne journée avec vos amis!'

### Before deploying the function
Open up the Jupyter shell and run the following <br>
pip install --upgrade nuclio-jupyter

In [14]:
%nuclio deploy -n nlp -p ai

[nuclio.deploy] 2019-11-16 06:45:40,553 (info) Building processor image
[nuclio.deploy] 2019-11-16 06:45:45,593 (info) Build complete
[nuclio.deploy] 2019-11-16 06:45:49,626 (info) Function deploy complete
[nuclio.deploy] 2019-11-16 06:45:49,635 done updating nlp, function address: 18.222.4.56:30450
%nuclio: function deployed


In [7]:
# Run a test with the new function. Replace the "function URL" with the actual URL:Port
# In order to get the function URL go to the Dashboard --> Functions --> drill down to project ai and then to the nlp function --> go to the Status tab 

!curl -X POST -d "Have a great day with your friends!" 18.222.4.56:30450

¡Que tengas un gran día con tus amigos!